In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install rasterio

     |████████████████████████████████| 19.3 MB 1.2 MB/s 


In [ ]:
import pandas as pd
from glob import glob
from osgeo import gdal
import os
import rasterio 
from rasterio.plot import show
from shutil import copyfile
from torch.utils.data import Dataset
from PIL import Image
import numpy as np
from numpy.random import permutation
import matplotlib.pyplot as plt

from torchvision import transforms
from torchvision.datasets import ImageFolder
from torchvision.models.inception import inception_v3
from torch.utils.data import DataLoader
from torch.autograd import Variable
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [ ]:
is_cuda = torch.cuda.is_available()
is_cuda

True

In [ ]:
def rmseLoss(output, target):
  loss_fn = nn.MSELoss()
  a = torch.sqrt(loss_fn(output.float(), target.float()))
  return a.float() 

In [ ]:
def Imshow(inp,cmap=None):
    """Imshow for Tensor."""
    inp = inp.numpy().transpose((1, 2, 0))
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    inp = std * inp + mean
    inp = np.clip(inp, 0, 1)
    plt.imshow(inp,cmap)
    
class LayerActivations():
    features=[]
    
    def __init__(self,model):
        self.features = []
        self.hook = model.register_forward_hook(self.hook_fn)
    
    def hook_fn(self,module,input,output):
        
        self.features.extend(output.view(output.size(0),-1).cpu().data)

    
    def remove(self):
        
        self.hook.remove()
    
class FeaturesDataset(Dataset):
    
    def __init__(self,featlst,labellst):
        self.featlst = featlst
        self.labellst = labellst
        
    def __getitem__(self,index):
        return (self.featlst[index],self.labellst[index])
    
    def __len__(self):
        return len(self.labellst)
    
def fit(epoch,model,data_loader,phase='training',volatile=False):
    if phase == 'training':
        model.train()
    if phase == 'validation':
        model.eval()
        volatile=True
    running_loss = 0.0
    running_correct = 0
    for  batch_idx , (data,target) in enumerate(data_loader):
        if is_cuda:
            data,target = data.cuda(),target.cuda()
            print(target)
        data , target = Variable(data,volatile),Variable(target)
        if phase == 'training':
            optimizer.zero_grad()
      
        output = model(data)
        loss = rmseLoss(output,target)
        
        running_loss += rmseLoss(output,target)
        preds = output.data.max(dim=1,keepdim=True)[1]
        running_correct += preds.eq(target.data.view_as(preds)).cpu().sum()
        if phase == 'training':
            loss.backward()
            optimizer.step()
    
    loss = float(running_loss)/float(len(data_loader.dataset))
    accuracy = 100. * float(running_correct)/float(len(data_loader.dataset))
    
    print(f'{phase} loss is {loss:{5}.{2}} and {phase} accuracy is {float(running_correct)}/{float(len(data_loader.dataset))}{accuracy:{10}.{4}}')
    return loss,accuracy

In [ ]:
len(trn_feat_loader.dataset)

72

In [ ]:
running_loss = 0.0
running_correct = 0
for  batch_idx , (data,target) in enumerate(trn_feat_loader):
        if is_cuda:
            data,target = data.cuda(),target.cuda()
        data , target = Variable(data),Variable(target)
        output = fc(data)
        loss = rmseLoss(output,target)
        preds = output.data.max(dim=1,keepdim=True)[1]
        running_correct += preds.eq(target.data.view_as(preds)).cpu().sum()
        print(running_correct)
        running_loss += rmseLoss(output,target)
        print(output)
        print(target)

print(running_loss)

In [ ]:

train_losses , train_accuracy = [],[]
val_losses , val_accuracy = [],[]
for epoch in range(1,100):
    epoch_loss, epoch_accuracy = fit(epoch,fc,trn_feat_loader,phase='training')
    val_epoch_loss , val_epoch_accuracy = fit(epoch,fc.eval(),val_feat_loader,phase='validation')
    train_losses.append(epoch_loss)
    train_accuracy.append(epoch_accuracy)
    val_losses.append(val_epoch_loss)
    val_accuracy.append(val_epoch_accuracy)

In [ ]:
## read tif image as Array : 

def tif_to_matrix(path):
  print(path)
  src_ds = gdal.Open(path)
  src_ds = src_ds.ReadAsArray()
  return src_ds 

In [ ]:
## read tif image as Array : 

def tif_stay_image(path):
  print(path)
  src_ds = rasterio.open(path)
  show(src_ds)
  return src_ds 

In [ ]:
# Normalize bands into 0.0 - 1.0 scale
def normalize(array):
    array_min, array_max = array.min(), array.max()
    return (array - array_min) / (array_max - array_min)

In [ ]:
## Create a Customized dataset class for Pytorch : 

class CustomDataSet(Dataset):
    def __init__(self, csv_file, root_dir, transform=None):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.landmarks_frame = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform

        
    def __len__(self):
        return len(self.landmarks_frame)
        

    def __getitem__(self, idx):

      if torch.is_tensor(idx):
        idx = idx.tolist()

      img_name = os.path.join(self.root_dir,
                                self.landmarks_frame.iloc[idx, -1])
      
      image = tif_to_matrix(img_name).astype(float)
      print(image.shape)

      soilreadings = self.landmarks_frame.iloc[idx, 1]
      soilreadings = np.array(soilreadings.astype(float))
      #print(soilreadings)
      sample = {'image': image, 'soilreadings': soilreadings}
     
      if self.transform:
          sample = self.transform(sample)

      return sample

In [ ]:
class ToTensor(object):
    """Convert ndarrays in sample to Tensors."""

    def __call__(self, sample):
        image, soilreadings = sample['image'], sample['soilreadings']

        # swap color axis because
        # numpy image: H x W x C
        # torch image: C X H X W
        image = image.transpose((2, 0, 1))
        return {'image': torch.from_numpy(image),
                'soilreadings': torch.from_numpy(soilreadings)}

In [ ]:
img_pil= tif_to_matrix('/content/drive/MyDrive/soilMoisture/Sentinel_images/20190410T112119_20190410T112212_T29TQF.tif')
x=normalize(img_pil)
x = torch.from_numpy(np.array(x.astype('float32')))

/content/drive/MyDrive/soilMoisture/Sentinel_images/20190410T112119_20190410T112212_T29TQF.tif


In [ ]:
class my_normalize(object):
    def __init__(self, mean, std):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """

        self.mean = mean
        self.std = std

    def __call__(self, sample) :
      
        tensor = sample['image']
        dtype = tensor.dtype
        self.mean = torch.as_tensor(self.mean, dtype=dtype, device=tensor.device)
        self.std = torch.as_tensor(self.std, dtype=dtype, device=tensor.device)
        if (self.std == 0).any():
            raise ValueError('std evaluated to zero after conversion to {}, leading to division by zero.'.format(dtype))
        if self.mean.ndim == 1:
            self.mean = self.mean.view(-1, 1, 1)
        if self.std.ndim == 1:
            self.std = self.std.view(-1, 1, 1)
        tensor.sub_(self.mean).div_(self.std)

        return {'image': tensor, 'soilreadings': sample['soilreadings']}


In [ ]:
from skimage.transform import resize

In [ ]:
class Rescale(object):
    """Rescale the image in a sample to a given size.

    Args:
        output_size (tuple or int): Desired output size. If tuple, output is
            matched to output_size. If int, smaller of image edges is matched
            to output_size keeping aspect ratio the same.
    """

    def __init__(self, output_size):
        assert isinstance(output_size, (int, tuple))
        self.output_size = output_size

    def __call__(self, sample):
        image, soilreadings = sample['image'], sample['soilreadings']

        h, w = image.shape[1:]
        
        if isinstance(self.output_size, int):
            if h > w:
                new_h, new_w = self.output_size * h / w, self.output_size
            else:
                new_h, new_w = self.output_size, self.output_size * w / h
        else:
            new_h, new_w = self.output_size

        new_h, new_w = int(new_h), int(new_w)

        image = image.transpose((1, 2, 0))
        img = resize(image, (new_h, new_w))
        
        # h and w are swapped for soilreadings because for images,
        # x and y axes are axis 1 and 0 respectively
        #soilreadings = soilreadings * [new_w / w, new_h / h]

        return {'image': img, 'soilreadings': soilreadings}

In [ ]:
MEAN = [0.485, 0.456, 0.406,0.485, 0.456, 0.406,0.485, 0.456, 0.406,0.485, 0.456, 0.406]
STD = [0.229, 0.224, 0.225,0.229, 0.224, 0.225,0.229, 0.224, 0.225,0.229, 0.224, 0.225]

data_transform = transforms.Compose([
        
        Rescale((299,299)),
        ToTensor(),
        my_normalize(MEAN,STD)  
    ])

In [ ]:
train_dataset = CustomDataSet(csv_file='/content/drive/MyDrive/Datasets/DataTrain.csv', 
                                    root_dir='/content/drive/MyDrive/soilMoisture/Sentinel_images',
                                           transform=data_transform)

validation_dataset = CustomDataSet(csv_file='/content/drive/MyDrive/Datasets/DataTest', 
                                    root_dir='/content/drive/MyDrive/soilMoisture/Images_test',
                                           transform=data_transform)

In [ ]:
train_dataset[2][0]

/content/drive/MyDrive/soilMoisture/Sentinel_images/20190410T112119_20190410T112212_T29TQF.tif
(12, 51, 46)


KeyError: ignored

In [ ]:
train_loader = DataLoader(train_dataset,batch_size=6,shuffle=False,num_workers=2)
validation_loader = DataLoader(validation_dataset,batch_size=3,shuffle=False,num_workers=2)

In [ ]:
import torchvision.models.inception
my_inception = inception_v3(pretrained=True)
my_inception.aux_logits = False

Downloading: "https://download.pytorch.org/models/inception_v3_google-0cc3c7bd.pth" to /root/.cache/torch/hub/checkpoints/inception_v3_google-0cc3c7bd.pth


In [ ]:
help(torchvision.models.inception.inception_v3)

Help on function inception_v3 in module torchvision.models.inception:

inception_v3(pretrained: bool = False, progress: bool = True, **kwargs: Any) -> 'Inception3'
    Inception v3 model architecture from
    `"Rethinking the Inception Architecture for Computer Vision" <http://arxiv.org/abs/1512.00567>`_.
    
    .. note::
        **Important**: In contrast to the other models the inception_v3 expects tensors with a size of
        N x 3 x 299 x 299, so ensure your images are sized accordingly.
    
    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
        progress (bool): If True, displays a progress bar of the download to stderr
        aux_logits (bool): If True, add an auxiliary branch that can improve training.
            Default: *True*
        transform_input (bool): If True, preprocesses the input according to the method with which it
            was trained on ImageNet. Default: *False*



In [ ]:
imy_inception = inception_v3(pretrained=True)
my_inception.aux_logits = False
my_inception

Inception3(
  (Conv2d_1a_3x3): BasicConv2d(
    (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_2a_3x3): BasicConv2d(
    (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_2b_3x3): BasicConv2d(
    (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (maxpool1): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  (Conv2d_3b_1x1): BasicConv2d(
    (conv): Conv2d(64, 80, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(80, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_4a_3x3): BasicConv2d(
    (conv): Conv2d(80, 192, kernel_size=(3, 3), stri

In [ ]:
if is_cuda:
    my_inception = my_inception.cuda()

In [ ]:
!pip install torchsummary

In [ ]:
from torchsummary import summary

summary(my_inception, (3, 299, 299))

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 32, 149, 149]             864
       BatchNorm2d-2         [-1, 32, 149, 149]              64
       BasicConv2d-3         [-1, 32, 149, 149]               0
            Conv2d-4         [-1, 32, 147, 147]           9,216
       BatchNorm2d-5         [-1, 32, 147, 147]              64
       BasicConv2d-6         [-1, 32, 147, 147]               0
            Conv2d-7         [-1, 64, 147, 147]          18,432
       BatchNorm2d-8         [-1, 64, 147, 147]             128
       BasicConv2d-9         [-1, 64, 147, 147]               0
        MaxPool2d-10           [-1, 64, 73, 73]               0
           Conv2d-11           [-1, 80, 73, 73]           5,120
      BatchNorm2d-12           [-1, 80, 73, 73]             160
      BasicConv2d-13           [-1, 80, 73, 73]               0
           Conv2d-14          [-1, 192,

In [ ]:
my_inception.Mixed_7b

InceptionE(
  (branch1x1): BasicConv2d(
    (conv): Conv2d(1280, 320, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(320, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (branch3x3_1): BasicConv2d(
    (conv): Conv2d(1280, 384, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(384, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (branch3x3_2a): BasicConv2d(
    (conv): Conv2d(384, 384, kernel_size=(1, 3), stride=(1, 1), padding=(0, 1), bias=False)
    (bn): BatchNorm2d(384, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (branch3x3_2b): BasicConv2d(
    (conv): Conv2d(384, 384, kernel_size=(3, 1), stride=(1, 1), padding=(1, 0), bias=False)
    (bn): BatchNorm2d(384, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (branch3x3dbl_1): BasicConv2d(
    (conv): Conv2d(1280, 448, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(448, eps=0.001, mo

In [ ]:
# Create LayerActivations object to store the output of inception model at a particular layer.
trn_features = LayerActivations(my_inception.Mixed_7b)
trn_labels = []

In [ ]:
trn_features

In [ ]:
# Passing all the data through the model , as a side effect the outputs will get stored 
# in the features list of the LayerActivations object. 

for da,la in enumerate(train_loader):
    da=la['image']
    da=da[:,:3,:,:]
    la=la['soilreadings']
    da = da.type(torch.cuda.FloatTensor)
    la = la.type(torch.cuda.LongTensor)
    #print(da,la)
    _ = my_inception(Variable(da.cuda()))
    trn_labels.extend(la)
#trn_features.remove()

/content/drive/MyDrive/soilMoisture/Sentinel_images/20190402T110621_20190402T110826_T29TQF.tif
/content/drive/MyDrive/soilMoisture/Sentinel_images/20190425T112121_20190425T112417_T29TQF.tif
(12, 51, 46)
(12, 51, 46)
/content/drive/MyDrive/soilMoisture/Sentinel_images/20190412T110621_20190412T111313_T29TQF.tif
/content/drive/MyDrive/soilMoisture/Sentinel_images/20190430T112119_20190430T112939_T29TQF.tif
(12, 51, 46)
(12, 51, 46)
/content/drive/MyDrive/soilMoisture/Sentinel_images/20190410T112119_20190410T112212_T29TQF.tif
/content/drive/MyDrive/soilMoisture/Sentinel_images/20190427T110629_20190427T111242_T29TQF.tif
(12, 51, 46)
(12, 51, 46)
/content/drive/MyDrive/soilMoisture/Sentinel_images/20190420T112119_20190420T112254_T29TQF.tif
/content/drive/MyDrive/soilMoisture/Sentinel_images/20190505T112121_20190505T112327_T29TQF.tif
(12, 51, 46)
(12, 51, 46)
/content/drive/MyDrive/soilMoisture/Sentinel_images/20190415T112121_20190415T112619_T29TQF.tif
/content/drive/MyDrive/soilMoisture/Senti

AttributeError: ignored

In [ ]:
len(trn_labels)

In [ ]:
# Repeat the same process for validation dataset .

val_features = LayerActivations(my_inception.Mixed_7b)
val_labels = []
for da,la in enumerate(validation_loader):
    da=la['image']
    da=da[:,:3,:,:]
    la=la['soilreadings']
    da = da.type(torch.cuda.FloatTensor)
    la = la.type(torch.cuda.LongTensor)
    _ = my_inception(Variable(da.cuda()))
    val_labels.extend(la)
val_features.remove()

/content/drive/MyDrive/soilMoisture/Images_test/20200206T111231_20200206T111434_T29TQF.tif
/content/drive/MyDrive/soilMoisture/Images_test/20200224T112109_20200224T112421_T29TQF.tif
(12, 51, 46)
(12, 51, 46)
/content/drive/MyDrive/soilMoisture/Images_test/20200221T110959_20200221T111742_T29TQF.tif
/content/drive/MyDrive/soilMoisture/Images_test/20200219T112111_20200219T112435_T29TQF.tif
(12, 51, 46)
(12, 51, 46)
/content/drive/MyDrive/soilMoisture/Images_test/20200302T110849_20200302T111641_T29TQF.tif
/content/drive/MyDrive/soilMoisture/Images_test/20200216T111131_20200216T111437_T29TQF.tif
(12, 51, 46)
(12, 51, 46)
/content/drive/MyDrive/soilMoisture/Images_test/20200310T112111_20200310T112439_T29TQF.tif
/content/drive/MyDrive/soilMoisture/Images_test/20200226T111051_20200226T111642_T29TQF.tif
(12, 51, 46)
(12, 51, 46)
/content/drive/MyDrive/soilMoisture/Images_test/20200327T110651_20200327T111753_T29TQF.tif
/content/drive/MyDrive/soilMoisture/Images_test/20200307T110911_20200307T1114

In [ ]:
trn_features.features[0]

tensor([0., 0., 0.,  ..., 0., 0., 0.])

In [ ]:
trn_features.features[10].size()

torch.Size([131072])

In [ ]:
len(trn_features.features)

84

In [ ]:
#Dataset for pre computed features for train and validation data sets

trn_feat_dset = FeaturesDataset(trn_features.features,trn_labels)
val_feat_dset = FeaturesDataset(val_features.features,val_labels)

#Data loaders for pre computed features for train and validation data sets

trn_feat_loader = DataLoader(trn_feat_dset,batch_size=5,shuffle=True)
val_feat_loader = DataLoader(val_feat_dset,batch_size=2)

In [ ]:
classes = 1
class FullyConnectedModel(nn.Module):
    
    def __init__(self,in_size,out_size,training=True):
        super().__init__()
        self.fc = nn.Linear(in_size,out_size)

    def forward(self,inp):
        out = F.dropout(inp, training=self.training)
        out = self.fc(out)
        return out

# The size of the output from the selected convolution feature 
fc_in_size = 131072

fc = FullyConnectedModel(fc_in_size,classes)
if is_cuda:
    fc = fc.cuda()

In [ ]:
optimizer = optim.Adam(fc.parameters(),lr=0.01)

In [ ]:

train_losses , train_accuracy = [],[]
val_losses , val_accuracy = [],[]
for epoch in range(1,10):
    epoch_loss, epoch_accuracy = fit(epoch,fc,trn_feat_loader,phase='training')
    val_epoch_loss , val_epoch_accuracy = fit(epoch,fc.eval(),val_feat_loader,phase='validation')
    train_losses.append(epoch_loss)
    train_accuracy.append(epoch_accuracy)
    val_losses.append(val_epoch_loss)
    val_accuracy.append(val_epoch_accuracy)

AttributeError: ignored